<a href="https://colab.research.google.com/github/terrysu64/Machine-Learning-and-Data-Science-Projects/blob/main/feature_extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Transfer learning - feature extraction

### inspect data

In [3]:
import zipfile

!wget https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip

zip_ref = zipfile.ZipFile("10_food_classes_10_percent.zip")
zip_ref.extractall()
zip_ref.close()

--2022-07-29 20:58:20--  https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.202.128, 173.194.203.128, 74.125.199.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.202.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 168546183 (161M) [application/zip]
Saving to: ‘10_food_classes_10_percent.zip.1’

10_food_classes_10_ 100%[===================>] 160.74M   256MB/s    in 0.6s    

2022-07-29 20:58:20 (256 MB/s) - ‘10_food_classes_10_percent.zip.1’ saved [168546183/168546183]



In [5]:
import os 

for dirpath,dirname,filenames in os.walk("10_food_classes_10_percent"):
  print(dirpath,len(dirname),len(filenames))

10_food_classes_10_percent 2 0
10_food_classes_10_percent/test 10 0
10_food_classes_10_percent/test/grilled_salmon 0 250
10_food_classes_10_percent/test/hamburger 0 250
10_food_classes_10_percent/test/ramen 0 250
10_food_classes_10_percent/test/ice_cream 0 250
10_food_classes_10_percent/test/chicken_wings 0 250
10_food_classes_10_percent/test/pizza 0 250
10_food_classes_10_percent/test/fried_rice 0 250
10_food_classes_10_percent/test/sushi 0 250
10_food_classes_10_percent/test/chicken_curry 0 250
10_food_classes_10_percent/test/steak 0 250
10_food_classes_10_percent/train 10 0
10_food_classes_10_percent/train/grilled_salmon 0 75
10_food_classes_10_percent/train/hamburger 0 75
10_food_classes_10_percent/train/ramen 0 75
10_food_classes_10_percent/train/ice_cream 0 75
10_food_classes_10_percent/train/chicken_wings 0 75
10_food_classes_10_percent/train/pizza 0 75
10_food_classes_10_percent/train/fried_rice 0 75
10_food_classes_10_percent/train/sushi 0 75
10_food_classes_10_percent/train/c

In [7]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator as IDG

#preprocess data
train_datagen = IDG(rescale=1./225,
                    rotation_range=0.2,
                    zoom_range=0.2,
                    horizontal_flip=True)

valid_datagen = IDG(rescale=1./225)

#paths
train_dir = "10_food_classes_10_percent/train"
test_dir = "10_food_classes_10_percent/test"

#import data and turn to batches
train_data = train_datagen.flow_from_directory(directory=train_dir,
                                               batch_size=32,
                                               target_size=(225,225),
                                               class_mode="categorical",
                                               seed=42)

valid_data = valid_datagen.flow_from_directory(directory=test_dir,
                                               batch_size=32,
                                               target_size=(225,225),
                                               class_mode="categorical",
                                               seed=42)

Found 750 images belonging to 10 classes.
Found 2500 images belonging to 10 classes.


### setting up callbacks

e.g:
* tracking experiments via TensorBoard
* Model checkpoint 
* Stopping model before overfitting

In [9]:
#tensorboard
import datetime
import tensorflow as tf

def tensorboard_callback(dir_name, exp_name):
  log_dir = f"{dir_name}/{exp_name}/{datetime.datetime.now().strftime('%Y%m%d-%H%M%S')}"
  callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir)
  print(f"saving tensorboard to: {log_dir}")
  return callback